# Learning Seattle's Work Habits from Bicycle Counts Data

This notebook originally appeared as a [post](http://jakevdp.github.io/blog/2015/07/23/learning-seattles-work-habits-from-bicycle-counts/) by Jake VanderPlas on the blog [Pythonic Perambulations](http://jakevdp.github.io) and [the related video series](http://jakevdp.github.io/blog/2017/03/03/reproducible-data-analysis-in-jupyter/)  Also adapted from by Amit Aides's "Intro session to data science with Python". 



The content is MIT licensed.*

## The Data

The data we will use here are the hourly bicycle counts on Seattle's Fremont Bridge. 

A time series is any data set where the values are measured at different points in time.

### Exercise 1: 

Understand the bike data.  Learn how to use curl command to download data. 


In [ ]:
#!curl -o FremontBridge.csv https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD

In [ ]:
import pandas as pd
data = pd.read_csv('FremontBridge12-16.csv', index_col='Date', parse_dates=True)
data.head()

In [ ]:
data.tail()

In [ ]:
type(data.index)

We'll do some quick data cleaning: we'll rename the columns to the shorter "West" and "East", set any missing values to zero, and add a "Total" column:

In [ ]:
data.columns = ['West', 'East']
data.fillna(0, inplace=True)

### Exercise 2: 

Add a new column of total which is the sum of east and west bike count. 

In [ ]:
# to do:
# data['Total'] = 
data.head()

## Visualize the Data

With matplotlib and seaborn, we can visualize our Pandas time series data. 

In [ ]:
# first some standard imports
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; 
seaborn.set()  # plot styling
seaborn.set_context(font_scale=1.5)
import numpy as np

In [ ]:
data.resample('W').sum().plot(figsize=(8, 8))
plt.ylabel('weekly trips');

### Exercise 3: 

Since the data is a time serie and is indexed by a timestamp, we can resample the data at a different frequency. Now resample the data to have the monthly total.  Plot out results. 

In [ ]:
# to do: 
# data.resample ... 

Group the data by the 12 monthes and plot out the mean.  

In [ ]:
data.groupby(data.index.month).mean().plot(figsize=(8, 8));

### Exercise 4: 

Group the data by the time (24 hours) and plot out the mean.  

In [ ]:
# to do: 


## Unsupervised Learning Extracting: Knowledge from the Data

From here, we could do a variety of other visualizations based on our intuition about what might affect bicycle counts. And we could also proceed by letting the dataset speak for itself, and use *unsupervised machine learning* techniques (that is, machine learning without reference to data labels) to learn what the data have to tell us.

We will consider each day in the dataset as a sample. 
For each day, we have 48 observations: two observations (east and west sidewalk sensors) for each of the 24 hour-long periods. The goal is to find out how what differen kinds of days are there relavent to the bike traffic. 

## Transforming the Data

In [ ]:
pivoted = data.pivot_table(['East', 'West'],
                           index=data.index.date,
                           columns=data.index.hour)
pivoted.head()

First let's check the first 5 records from the East sidewalk counter. 

In [ ]:
first_five_east = pivoted.iloc[:5, :24]
first_five_east

### Exercise 5: 
In the pivot table, slice the first 5 records from the West bound traffic. 

In [ ]:
# to do:
# first_five_west ...

Now we try to figure out some patterns from the data. Let's start with the first five east bound records. 

In [ ]:
first_five_east.plot(figsize=(8, 8))

Let's try transpose the data.  

In [ ]:
first_five_east.T.plot(figsize=(8, 8))

Now we can plot the data collected east bound for all the dates 

In [ ]:
east = pivoted.iloc[:, :24]
east.T.plot(legend=False, alpha=0.01, figsize=(8, 8))

### Exercise 6:

Plot all the records from the West bound counter for all the dates. 

In [ ]:
# to do:
# west = ...

These plots give us some insight into the data. It looks like there are two types of clusters: the first cluster shows a sharp bimodal traffic pattern, while the second shows a wide unimodal pattern. In the bimodal cluster, we see a peak in the morning and another peak in the afternoon which is clearly a commute pattern. 


## Dimensionality Reduction

We can think of this data now as representing more than 1000 distinct objects which live in a *48-dimensional* space: the value of each dimension is the number of bicycle trips measured on a particular side of the bridge at a particular hour.
Visualizing 48-dimensional data is quite difficult, so instead we will use a standard *dimensionality reduction* technique to project this to a more manageable size.

The technique we'll use is [Principal Component Analysis (PCA)](http://scikit-learn.org/stable/modules/decomposition.html), a fast linear projection which rotates the data such that the projection preserves the maximum variance.
We can ask for components preserving 90% of the variance as follows:

We first extract the raw values and put them in a matrix:

In [ ]:
X = pivoted.values
X.shape

In [ ]:
from sklearn.decomposition import PCA

#Xpca = PCA(n_components=2).fit_transform(X)
Xpca = PCA(0.90, svd_solver='full').fit_transform(X)
Xpca.shape

The output has two dimensions, which means that these two projected components describe at least 90% of the total variance in the dataset.
While 48-dimensional data is difficult to plot, we certainly know how to plot two-dimensional data: we'll do a simple scatter plot, and for reference we'll color each point according to the total number of trips taken that day:

## Exercise 7:

In this PCA analysis above, how many projected components are needed to describe at least 95% of the total variance in the dataset? 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
plt.scatter(Xpca[:,0], Xpca[:, 1])

In [ ]:
total_trips = X.sum(1)

plt.figure(figsize=(8, 8))
plt.scatter(
    Xpca[:, 0],
    Xpca[:, 1],
    c=total_trips,
    cmap='jet'
)

plt.colorbar(label='total trips');

We see that the days lie in two quite distinct groups, and that the total number of trips increases along the length of each projected cluster.
Further, the two groups begin to be less distinguishable when the number of trips during the day is very small.

I find this extremely interesting: from the raw data, we can determine that there are basically *two primary types of days* for Seattle bicyclists.
Let's model these clusters and try to figure out what these types-of-day are.

## Unsupervised Clustering

When you have groups of data you'd like to automatically separate, but no previously-determined labels for the groups, the type of algorithm you are looking at is a *clustering* algorithm.
There are a number of clustering algorithms out there, but for nicely-defined oval-shaped blobs like we see above, [Gaussian Mixture Models](http://scikit-learn.org/stable/modules/mixture.html) are a very good choice.
We can compute the Gaussian Mixture Model of the data using, again, scikit-learn, and quickly plot the predicted labels for the points:

In [ ]:
from sklearn.mixture import GaussianMixture 
gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=0)
gmm.fit(Xpca)

In [ ]:
cluster_label = gmm.predict(Xpca)
cluster_label

In [ ]:
plt.figure(figsize=(8, 8))

for i, color in enumerate(['black', 'red']):
    inds = cluster_label == i
    x, y = Xpca[inds, 0], Xpca[inds, 1]
    plt.scatter(x, y, c=color, label="Cluster {}".format(i),
               edgecolors='none')

plt.legend()

This clustering seems to have done the job, and separated the two groups we are interested in.
Let's join these inferred cluster labels to the initial dataset:

In [ ]:
pivoted['Cluster'] = cluster_label
data = data.join(pivoted['Cluster'], on=data.index.date)
data.head()

Now we can find the average trend by cluster and time using a GroupBy within this updated dataset

In [ ]:
by_hour = data.groupby(['Cluster', data.index.time]).mean()

In [ ]:
by_hour.head()

In [ ]:
by_hour.tail()

Finally, we can plot the average hourly trend among the days within each cluster:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
hourly_ticks = 4 * 60 * 60 * np.arange(6)

for i, ax in enumerate(axes.flatten()[:2]):
    by_hour.loc[i].plot(y="West", ax=ax, xticks=hourly_ticks)
    by_hour.loc[i].plot(y="East", ax=ax, xticks=hourly_ticks)
    ax.set_title('Cluster {0}'.format(i))
    ax.set_ylabel('average hourly trips')

These plots give us some insight into the interpretation of the two clusters: the first cluster shows a sharp bimodal traffic pattern, while the second shows a wide unimodal pattern.

In the bimodal cluster, we see a peak around 8:00am which is dominated by cyclists on the west sidewalk, and another peak around 5:00pm which is dominated by cyclists on the east sidewalk.
This is very clearly a commute pattern, with the majority of cyclists riding toward downtown Seattle in the morning, and away from downtown Seattle in the evening.

In the unimodal cluster, we see fairly steady traffic in each direction beginning early in the morning and going until late at night, with a peak around 2:00 in the afternoon.
This is very clearly a recreational pattern of use, with people out riding through the entire day.

This is quite fascinating: from simple unsupervised dimensionality reduction and clustering, we've discovered two distinct classes of days in the data, and found that these classes have very intuitive explanations.

## Seattle's Work Habits

Let's go one step deeper and figure out what we can learn about people (well, bicycle commuters) in Seattle from just this hourly commute data.
As a rough approximation, you might guess that these two classes of data might be largely reflective of workdays in the first cluster, and non-work days in the second.
We can check this intuition by re-plotting our projected data, except labeling them by day of the week:

In [ ]:
dayofweek = pd.to_datetime(pivoted.index).dayofweek

plt.figure(figsize=(8, 8))
plt.scatter(Xpca[:, 0], Xpca[:, 1], c=dayofweek,
            cmap=plt.cm.get_cmap('jet', 7))
cb = plt.colorbar(ticks=range(7))
cb.set_ticklabels(['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun'])
plt.clim(-0.5, 6.5);

We see that the weekday/weekend intuition holds, but only to a point: in particular, it is clear that **there are a handful of weekdays which follow the typical weekend pattern!**
Further, it's interesting to note that Fridays tend to be pulled closer to weekend days in this plot, though as a whole they still fall solidly in the work-day cluster.

Let's take a closer look at the "special" weekdays that fall in the "wrong" cluster.
We start by constructing a dataset listing the cluster id and the day of the week for each of the dates in our dataset:

In [ ]:
results = pd.DataFrame({'cluster': cluster_label,
                        'is_weekend': (dayofweek > 4),
                        'weekday': pivoted.index.map(lambda x: x.strftime('%a'))},
                       index=pivoted.index)
results.head()

In [ ]:
results[results["cluster"]==0]

In [ ]:
results[results["cluster"]==1]

In [ ]:
weekend_workdays = results.query('cluster == 0 and is_weekend')
len(weekend_workdays)

Apparently, there is not a single weekend during the year where Seattle cyclists as a whole decide to go to work.  Similarly, we can see how many weekdays fall in the second, recreation-oriented cluster:

In [ ]:
midweek_holidays = results.query('cluster == 1 and not is_weekend')
len(midweek_holidays)

In [ ]:
midweek_holidays

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', '2016', return_name=True)
holidays

Just for completeness, we will add to the list the day before and day after each of these holidays:

In [ ]:
holidays_all = pd.concat([holidays,
                         "Day Before " + holidays.shift(-1, 'D'),
                         "Day After " + holidays.shift(1, 'D')])
holidays_all = holidays_all.sort_index()
holidays_all.head()